# Coursera Peer Graded Assignment - Capstone Project

## Introduction

Bangalore, officially Bengaluru, is the capital of the Indian state of Karnataka. It has a population of over ten million, making it a megacity and the third-most populous city and fifth-most populous urban agglomeration in India. 

Bengaluru is widely regarded as the "Silicon Valley of India" (or "IT capital of India") because of its role as the nation's leading information technology (IT) exporter. Indian technological organisations such as ISRO, Infosys, Wipro and HAL are headquartered in the city. A demographically diverse city, Bangalore is the second fastest-growing major metropolis in India. Recent estimates of the metro economy of its urban area have ranked Bangalore either the fourth or fifth-most productive metro area of India. It is home to many educational and research institutions in India, such as Indian Institute of Science (IISc), Indian Institute of Management (Bangalore) (IIMB), International Institute of Information Technology, Bangalore (IIITB), National Institute of Fashion Technology, Bangalore, National Institute of Design, Bangalore (NID R&D Campus), National Law School of India University (NLSIU) and National Institute of Mental Health and Neurosciences (NIMHANS). Numerous state-owned aerospace and defence organisations, such as Bharat Electronics, Hindustan Aeronautics and National Aerospace Laboratories are located in the city. The city also houses the Kannada film industry.

## Business Problem

With huge development in the city, the population is expected to increase in the coming years. Bangalore's 2020 population is now estimated at 12,326,532. The aim of this project is to find suitable places in the city with best dining experiences.

## Target Audience

The target audience of this report is for anyone who wants to open up a new restaurant in Bangalore.

## Data

The data for this project has been taken from the Wikipedia webpage. Where the list of all neighbohoods in every regions has been listed. 

A link to the webpage is provided here: "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore"

### 1. Importing required libraries

We start this project by importing all the necessary libraries required.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### 2. Download and explore the Dataset

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore"
html = requests.get(url).content
df = pd.read_html(html)
print(df)

[                 Name  Image  \
0     Cantonment area    NaN   
1              Domlur    NaN   
2         Indiranagar    NaN   
3   Jeevanbheemanagar    NaN   
4         Malleswaram    NaN   
5           Pete area    NaN   
6      Sadashivanagar    NaN   
7       Seshadripuram    NaN   
8        Shivajinagar    NaN   
9              Ulsoor    NaN   
10      Vasanth Nagar    NaN   

                                              Summary  
0   The Cantonment area in Bangalore was used as a...  
1   Formerly part of the Cantonment area, Domlur h...  
2   Indiranagar is a sought-after residential and ...  
3                                                 NaN  
4                                                 NaN  
5   Established by Kempe Gowda I at the time of cr...  
6   Sadashivanagar is an upscale neighbourhood in ...  
7   Seshadripuram was established in 1892 to reduc...  
8   Shivajinagar is one of the older areas of the ...  
9   Ulsoor (or Halasuru) is one of the oldest area... 

As we see above, there are eight tables divided by their respective regions. In the next step, we will merge all the tables to form one full dataframe set.

In [3]:
df[0]["Region"] = "Central"
df[1]["Region"] = "Eastern"
df[2]["Region"] = "North Eastern"
df[3]["Region"] = "Northern"
df[4]["Region"] = "South Eastern"
df[5]["Region"] = "Southern"
df[6]["Region"] = "Southern suburbs"
df[7]["Region"] = "Western"

In [4]:
bangalore_data = pd.concat([df[0] , df[1] , df[2] , df[3] , df[4] , df[5] , df[6] , df[7]] , ignore_index = True)

We only need the list of neighborhood names. So we drop the unnecessary columns such as "Image" and "Summary"

In [5]:
bangalore_data = bangalore_data.drop(columns = ["Image" , "Summary"])
print(bangalore_data)

                    Name            Region
0        Cantonment area           Central
1                 Domlur           Central
2            Indiranagar           Central
3      Jeevanbheemanagar           Central
4            Malleswaram           Central
5              Pete area           Central
6         Sadashivanagar           Central
7          Seshadripuram           Central
8           Shivajinagar           Central
9                 Ulsoor           Central
10         Vasanth Nagar           Central
11             Bellandur           Eastern
12        CV Raman Nagar           Eastern
13                 Hoodi           Eastern
14      Krishnarajapuram           Eastern
15          Mahadevapura           Eastern
16          Marathahalli           Eastern
17               Varthur           Eastern
18            Whitefield           Eastern
19             Banaswadi     North Eastern
20            HBR Layout     North Eastern
21              Horamavu     North Eastern
22         

### 3. Adding latitudes and longitudes for the neighborhood names

In [6]:
locator = Nominatim(user_agent="myGeocoder")

from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# 2- - create location column
bangalore_data['location'] = bangalore_data['Name'].apply(geocode)

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
bangalore_data['point'] = bangalore_data['location'].apply(lambda loc: tuple(loc.point) if loc else None)

print(bangalore_data)

                    Name            Region  \
0        Cantonment area           Central   
1                 Domlur           Central   
2            Indiranagar           Central   
3      Jeevanbheemanagar           Central   
4            Malleswaram           Central   
5              Pete area           Central   
6         Sadashivanagar           Central   
7          Seshadripuram           Central   
8           Shivajinagar           Central   
9                 Ulsoor           Central   
10         Vasanth Nagar           Central   
11             Bellandur           Eastern   
12        CV Raman Nagar           Eastern   
13                 Hoodi           Eastern   
14      Krishnarajapuram           Eastern   
15          Mahadevapura           Eastern   
16          Marathahalli           Eastern   
17               Varthur           Eastern   
18            Whitefield           Eastern   
19             Banaswadi     North Eastern   
20            HBR Layout     North

In [7]:
bangalore_data = bangalore_data.dropna(axis = 0)
print(bangalore_data)

                    Name            Region  \
0        Cantonment area           Central   
1                 Domlur           Central   
2            Indiranagar           Central   
4            Malleswaram           Central   
5              Pete area           Central   
6         Sadashivanagar           Central   
7          Seshadripuram           Central   
8           Shivajinagar           Central   
9                 Ulsoor           Central   
10         Vasanth Nagar           Central   
11             Bellandur           Eastern   
12        CV Raman Nagar           Eastern   
13                 Hoodi           Eastern   
14      Krishnarajapuram           Eastern   
15          Mahadevapura           Eastern   
16          Marathahalli           Eastern   
17               Varthur           Eastern   
18            Whitefield           Eastern   
19             Banaswadi     North Eastern   
20            HBR Layout     North Eastern   
21              Horamavu     North

In [8]:
bangalore_data["latitudes"] = [item[0] for item in bangalore_data["point"]]
bangalore_data["longitudes"] = [item[1] for item in bangalore_data["point"]]
print(bangalore_data)

                    Name            Region  \
0        Cantonment area           Central   
1                 Domlur           Central   
2            Indiranagar           Central   
4            Malleswaram           Central   
5              Pete area           Central   
6         Sadashivanagar           Central   
7          Seshadripuram           Central   
8           Shivajinagar           Central   
9                 Ulsoor           Central   
10         Vasanth Nagar           Central   
11             Bellandur           Eastern   
12        CV Raman Nagar           Eastern   
13                 Hoodi           Eastern   
14      Krishnarajapuram           Eastern   
15          Mahadevapura           Eastern   
16          Marathahalli           Eastern   
17               Varthur           Eastern   
18            Whitefield           Eastern   
19             Banaswadi     North Eastern   
20            HBR Layout     North Eastern   
21              Horamavu     North

In [9]:
bangalore_data = bangalore_data.drop(columns = ["location" , "point"])
print(bangalore_data)

                    Name            Region  latitudes  longitudes
0        Cantonment area           Central   8.468109  -13.250525
1                 Domlur           Central  12.962467   77.638196
2            Indiranagar           Central  12.973291   77.640467
4            Malleswaram           Central  16.702904   81.787929
5              Pete area           Central  48.449042 -124.100218
6         Sadashivanagar           Central  13.007708   77.579589
7          Seshadripuram           Central  12.993188   77.575342
8           Shivajinagar           Central  18.532249   73.849960
9                 Ulsoor           Central  12.977879   77.624670
10         Vasanth Nagar           Central  12.988721   77.585169
11             Bellandur           Eastern  12.936445   77.667572
12        CV Raman Nagar           Eastern  19.060047   72.923752
13                 Hoodi           Eastern  12.991903   77.716201
14      Krishnarajapuram           Eastern  13.000509   77.675394
15        

### 4. Use geopy library to get the latitude and longitude values of Bangalore City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent bangalore_explorer, as shown below.

In [10]:
address = 'Bangalore, Karnataka'

geolocator = Nominatim(user_agent="bangalore_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore City are 12.9791198, 77.5912997.


### 5. Create a map of Bangalore city with neighborhoods superimposed on top

In [11]:
# create map of Bangalore using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, region, name in zip(bangalore_data['latitudes'], bangalore_data['longitudes'], bangalore_data['Region'], bangalore_data['Name']):
    label = '{}, {}'.format(name, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_bangalore) 
                                                                                                                                                                                                                                               
                                                                                                                                                                                                                                                                                 
map_bangalore                                             

### 6. Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'SHSKPYGRDE1TC11LSGJQ0RBRS4ENYMIKMFUBWO5EJVF5WF1N' 
CLIENT_SECRET = 'URS3VNDHHSX2AECQWMUA3SGYSCSQFBNZFGAYTGDFJ1D0CNP3' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SHSKPYGRDE1TC11LSGJQ0RBRS4ENYMIKMFUBWO5EJVF5WF1N
CLIENT_SECRET:URS3VNDHHSX2AECQWMUA3SGYSCSQFBNZFGAYTGDFJ1D0CNP3


In [13]:
def getNearbyVenues(names , latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
LIMIT = 20

bangalore_venues = getNearbyVenues(names = bangalore_data['Name'],
                                   latitudes = bangalore_data['latitudes'],
                                   longitudes = bangalore_data['longitudes'])

Cantonment area
Domlur
Indiranagar
Malleswaram
Pete area
Sadashivanagar
Seshadripuram
Shivajinagar
Ulsoor
Vasanth Nagar
Bellandur
CV Raman Nagar
Hoodi
Krishnarajapuram
Mahadevapura
Marathahalli
Varthur
Whitefield
Banaswadi
HBR Layout
Horamavu
Kalyan Nagar
Kammanahalli
Lingarajapuram
Ramamurthy Nagar
Hebbal
Jalahalli
Mathikere
Peenya
R. T. Nagar
Vidyaranyapura
Yelahanka
Yeshwanthpur
Bommanahalli
Bommasandra
BTM Layout
Electronic City
HSR Layout
Koramangala
Madiwala
Banashankari
Basavanagudi
Girinagar
J. P. Nagar
Jayanagar
Kumaraswamy Layout
Padmanabhanagar
Uttarahalli
Anjanapura
Arekere
Begur
Gottigere
Hulimavu
Kothnur
Basaveshwaranagar
Kamakshipalya
Kengeri
Mahalakshmi Layout
Nagarbhavi
Nandini Layout
Nayandahalli
Rajajinagar
Rajarajeshwari Nagar
Vijayanagar


In [15]:
print(bangalore_venues.shape)
bangalore_venues.head()

(473, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cantonment area,8.468109,-13.250525,LACS villa,8.470022,-13.250423,Bed & Breakfast
1,Domlur,12.962467,77.638196,Lavonne,12.963909,77.638579,Café
2,Domlur,12.962467,77.638196,Smoke House Deli,12.965584,77.641498,Deli / Bodega
3,Domlur,12.962467,77.638196,Barbeque Nation,12.962684,77.641599,BBQ Joint
4,Domlur,12.962467,77.638196,Drops Total Spirits,12.964527,77.641636,Liquor Store


Let's check how many venues were returned for each neighborhood

In [16]:
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BTM Layout,20,20,20,20,20,20
Banaswadi,7,7,7,7,7,7
Basaveshwaranagar,17,17,17,17,17,17
Begur,20,20,20,20,20,20
Bommanahalli,4,4,4,4,4,4
Bommasandra,2,2,2,2,2,2
CV Raman Nagar,3,3,3,3,3,3
Cantonment area,1,1,1,1,1,1
Domlur,19,19,19,19,19,19


Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 132 uniques categories.


### 7. Analyzing each Neighborhood

In [18]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bakery,Balinese Restaurant,Bar,Bed & Breakfast,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Station,Business Service,Café,Candy Store,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dry Cleaner,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,Golf Course,Gourmet Shop,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Paintball Field,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Residential Building (Apartment / Condo),Restaurant,Road,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Tapas Restaurant,Train Station,Udon Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Cantonment area,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Domlur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Domlur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Domlur,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Domlur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
bangalore_onehot.shape

(473, 133)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bakery,Balinese Restaurant,Bar,Bed & Breakfast,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Station,Business Service,Café,Candy Store,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dry Cleaner,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Gastropub,Golf Course,Gourmet Shop,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multiplex,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Paintball Field,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Residential Building (Apartment / Condo),Restaurant,Road,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Tapas Restaurant,Train Station,Udon Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,BTM Layout,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.00,0.0000,0.0,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.0,0.00,0.00,0.100000,0.00,0.000000,0.0,0.050000,0.000000,0.000000,0.0,0.000000,0.050000,0.000000,0.05,0.000000,0.000000,0.0,0.00,0.00,0.050000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.150000,0.000000,0.100000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.050000,0.0,0.050000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.050000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.00,0.100000,0.000000,0.000000,0.000000
1,Banaswadi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.0,0.000000,0.142857,0.00,0.0000,0.0,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.00,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.285714,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.00,0.285714,0.000000,0.000000,0.000000
2,Basaveshwaranagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.058824,0.00,0.0000

In [21]:
num_top_venues = 5

for hood in bangalore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bangalore_grouped[bangalore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----BTM Layout----
                           venue  freq
0                 Ice Cream Shop  0.15
1                         Bakery  0.10
2              Indian Restaurant  0.10
3  Vegetarian / Vegan Restaurant  0.10
4             Chinese Restaurant  0.10


----Banaswadi----
                           venue  freq
0              Indian Restaurant  0.29
1  Vegetarian / Vegan Restaurant  0.29
2                         Bakery  0.14
3                      BBQ Joint  0.14
4                  Grocery Store  0.14


----Basaveshwaranagar----
                  venue  freq
0        Ice Cream Shop  0.18
1           Pizza Place  0.12
2                  Café  0.12
3  Fast Food Restaurant  0.12
4   Sporting Goods Shop  0.06


----Begur----
                      venue  freq
0  Mediterranean Restaurant  0.25
1              Cocktail Bar  0.10
2                     Hotel  0.10
3        Spanish Restaurant  0.10
4          Tapas Restaurant  0.10


----Bommanahalli----
               venue  freq
0        Auto G

Let's put that into a *pandas* dataframe. First, let's write a function to sort the venues in descending order.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BTM Layout,Ice Cream Shop,Chinese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Indian Restaurant,Coffee Shop,Café,Department Store,Dim Sum Restaurant,Restaurant
1,Banaswadi,Vegetarian / Vegan Restaurant,Indian Restaurant,BBQ Joint,Grocery Store,Bakery,Yoga Studio,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant
2,Basaveshwaranagar,Ice Cream Shop,Pizza Place,Fast Food Restaurant,Café,Sporting Goods Shop,Liquor Store,Miscellaneous Shop,Department Store,Park,Bus Station
3,Begur,Mediterranean Restaurant,Cocktail Bar,Hotel,Tapas Restaurant,Spanish Restaurant,Scenic Lookout,Pub,Seafood Restaurant,Gastropub,Restaurant
4,Bommanahalli,Shopping Mall,Indian Restaurant,Bus Station,Auto Garage,Department Store,Electronics Store,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant


### 8. Cluster Neighborhood
Run k-means to cluster the neighborhood into 5 clusters.

In [24]:
# set number of clusters
kclusters = 5

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 4, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangalore_data = bangalore_data.rename(columns = {"Name" : "Neighborhood"})

bangalore_merged = bangalore_data

bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bangalore_merged = bangalore_merged.dropna(axis = 0)

bangalore_merged.head()

,Neighborhood,Region,latitudes,longitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cantonment area,Central,8.468109,-13.250525,4.0,Bed & Breakfast,Yoga Studio,Cocktail Bar,Coffee Shop,Convenience Store,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
1,Domlur,Central,12.962467,77.638196,3.0,Indian Restaurant,Café,Sandwich Place,Food & Drink Shop,Italian Restaurant,BBQ Joint,Liquor Store,Sports Bar,Rajasthani Restaurant,Pub
2,Indiranagar,Central,12.973291,77.640467,3.0,Lounge,Cocktail Bar,Pub,Cupcake Shop,Chocolate Shop,Asian Restaurant,Restaurant,Dessert Shop,Bakery,Indian Restaurant
6,Sadashivanagar,Central,13.007708,77.579589,3.0,Coffee Shop,Department Store,Café,Indian Restaurant,Chinese Restaurant,Dessert Shop,Spa,Boat or Ferry,Plaza,Gym
7,Seshadripuram,Central,12.993188,77.575342,3.0,Clothing Store,Coffee Shop,Indian Restaurant,Karnataka Restaurant,Department Store,Dessert Shop,Miscellaneous Shop,Restaurant,Donut Shop,Bar


Let's visualize the resulting cluster
    
    

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['latitudes'], bangalore_merged['longitudes'], bangalore_merged['Neighborhood'], bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster)-1],
        fill=True,
        fill_color= rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 9. Results

The neighborhoods are divided into 5 clusters. And the results are shown below.

#### Cluster 1

In [27]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Western,Gym,Yoga Studio,Deli / Bodega,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Cupcake Shop


#### Cluster 2

In [28]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Shivajinagar,Indian Restaurant,Hospital,Department Store,Hotel,Food Truck,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
13,Hoodi,Indian Restaurant,Yoga Studio,Deli / Bodega,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Cupcake Shop
17,Varthur,Candy Store,Indian Restaurant,Yoga Studio,Department Store,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop,Cupcake Shop
30,R. T. Nagar,Indian Restaurant,Chinese Restaurant,Playground,Department Store,Deli / Bodega,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop
52,Gottigere,Indian Restaurant,Chinese Restaurant,Department Store,Deli / Bodega,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop,Cupcake Shop


#### Cluster 3

In [29]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Kammanahalli,Bar,Fast Food Restaurant,Cocktail Bar,Coffee Shop,Convenience Store,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


#### Cluster 4

In [30]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Domlur,Indian Restaurant,Café,Sandwich Place,Food & Drink Shop,Italian Restaurant,BBQ Joint,Liquor Store,Sports Bar,Rajasthani Restaurant,Pub
2,Indiranagar,Lounge,Cocktail Bar,Pub,Cupcake Shop,Chocolate Shop,Asian Restaurant,Restaurant,Dessert Shop,Bakery,Indian Restaurant
6,Sadashivanagar,Coffee Shop,Department Store,Café,Indian Restaurant,Chinese Restaurant,Dessert Shop,Spa,Boat or Ferry,Plaza,Gym
7,Seshadripuram,Clothing Store,Coffee Shop,Indian Restaurant,Karnataka Restaurant,Department Store,Dessert Shop,Miscellaneous Shop,Restaurant,Donut Shop,Bar
9,Ulsoor,Hotel,Light Rail Station,Indian Restaurant,Café,Department Store,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop
10,Vasanth Nagar,Hotel,Indian Restaurant,Coffee Shop,Hotel Bar,Sushi Restaurant,Golf Course,Lounge,Nightclub,Gym,Art Museum
12,CV Raman Nagar,Auto Garage,Indian Restaurant,Café,Yoga Studio,Dry Cleaner,Donut Shop,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
14,Krishnarajapuram,ATM,Fast Food Restaurant,Indian Restaurant,Train Station,Movie Theater,Deli / Bodega,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
15,Mahadevapura,Coffee Shop,Movie Theater,Shopping Mall,Multiplex,Clothing Store,Bar,Fast Food Restaurant,French Restaurant,Italian Restaurant,Convenience Store
16,Marathahalli,Indian Restaurant,Clothing Store,Burger Joint,Bakery,Kerala Restaurant,Bus Station,Café,Furniture / Home Store,Vegetarian / Vegan Restaurant,Lounge


#### Cluster 5

In [31]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4, bangalore_merged.columns[[0] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cantonment area,Bed & Breakfast,Yoga Studio,Cocktail Bar,Coffee Shop,Convenience Store,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


### 10. Discussion

After analysing the above cluster, we find that the fourth cluster fits best to answer our question. The top neighborhood places with Restaurants as the most common venue are Domlur, Marathahalli, Banaswadi, Horamavu and Kalyan Nagar.

These five places have restaurants of different cuisines as their top most common venue. The different cuisines being Indian, Chinese, Vegetarian and Korean. So depending upon the choice of cuisine, the client can easily choose the areas in which he/she would want to open a restaurant.

### 11. Conclusion

According to the report "The Rise of Restaurant Industry in India", published by AIMS, (link: "https://theaims.ac.in/resources/rise-of-the-restaurant-industry-in-india.html"), the food industry in India is rapidly expanding. The changing lifestyle, rise of the nuclear family, more women stepping out of their traditional roles to go out and work, rapid urbanization are some of the factors responsible for the growth of the restaurant industry in India. Added to that is the increased exposure to international lifestyles and cuisines. More and more Indians are demonstrating a growing appetite for a variety of cuisines ranging from Chinese and Italian to Mexican and Middle Eastern.

Greater awareness of global cuisines combined with a larger disposable income is leading many Indian consumers to seek experiential eating or fine dining. Fine dining is not just about going out and eating. Fine dining is about elevating the dining experience of consumers through ambience, décor, presentation of the food, quality of service,use of gourmet ingredients, etc. High-end or fine dining is slowly coming of age in India.

So, the people who are interested in opening a restaurant should not only prioritize the area, but also need to emphasize on the type of cuisine, quality of service, ambience of the restaurant. All these factors builds to providing a great dining experience.